Am ales un llm putin mai mare decat 780M, insa rezultatele nu au fost cu mult diferite

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "ai-forever/mGPT-1.3B-romanian"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/5.77G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.77G [00:00<?, ?B/s]

AM incercat un finetuning pe un set de date foarte foarte mic(3 cereri), rularea a durat in jurul a 2 ore

In [ ]:
from datasets import Dataset

data = [
    {"text": """### CERERE DE CHEMARE ÎN JUDECATĂ
Reclamant: Ion Popescu
Pârât: SC Exemplu SRL
Obiect: Recuperare sumă de 10.000 lei
Introducere: Subsemnatul Ion Popescu formulez prezenta cerere pentru recuperarea sumei datorate de SC Exemplu SRL.
Motivare: Pârâtul nu a achitat factura nr. 123/2025 în termenul legal.
Solicitări: Solicit instanței obligarea pârâtului la plata sumei de 10.000 lei, precum și a dobânzilor și a cheltuielilor de judecată."""},

    {"text": """### CERERE DE CHEMARE ÎN JUDECATĂ
Reclamant: Maria Ionescu
Pârât: SC AltEx SRL
Obiect: Plata daunelor de 5.000 lei
Introducere: Subsemnata Maria Ionescu formulez prezenta cerere pentru plata daunelor cauzate de SC AltEx SRL.
Motivare: Pârâtul a încălcat contractul de prestări servicii încheiat la data de 01.01.2025.
Solicitări: Solicit instanței obligarea pârâtului la plata sumei de 5.000 lei și a cheltuielilor de judecată."""},

    {"text": """### CERERE DE CHEMARE ÎN JUDECATĂ
Reclamant: Andrei Popa
Pârât: SC Servicii SRL
Obiect: Recuperare chirie restantă 3.000 lei
Introducere: Subsemnatul Andrei Popa formulez prezenta cerere pentru recuperarea chiriei restante.
Motivare: Pârâtul nu a achitat chiria aferentă lunilor ianuarie–martie 2025.
Solicitări: Solicit instanței obligarea pârâtului la plata sumei de 3.000 lei și a cheltuielilor de judecată."""}
]

dataset = Dataset.from_list(data)


In [ ]:
def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_fn, batched=True)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
def tokenize_and_prepare_labels(example):
    tokens = tokenizer(example["text"], truncation=True, max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()  # crucial!
    return tokens

tokenized_dataset = dataset.map(tokenize_and_prepare_labels, remove_columns=["text"])


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

model = get_peft_model(model, lora_config)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    report_to=[],
    fp16=True,  # dacă ai GPU compatibil
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)


/tmp/ipython-input-3144144177.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=3.196941693623861, metrics={'train_runtime': 10169.6105, 'train_samples_per_second': 0.0, 'train_steps_per_second': 0.0, 'total_flos': 3703136993280.0, 'train_loss': 3.196941693623861, 'epoch': 1.0})

Rezultatele nu au fost considerabil mai bune, executia tot nu a ajuns pana la final